In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [5]:
from sklearn.metrics import f1_score, precision_recall_curve


In [6]:
result_file = 'val_emdedding_results_047_97.330729_90.909091.txt'
fp = open(result_file)
lines = fp.readlines() 
lines = [x.strip() for x in lines]
fp.close()
print(lines[:10])

['val_covid_ct_scan_0,0,0,2.465393304824829,-2.1594154834747314,0,2.4482500553131104,-2.132641315460205,0,2.489469528198242,-2.1809892654418945,0,2.4387567043304443,-2.1386044025421143,0,2.2616212368011475,-2.122927665710449', 'val_covid_ct_scan_1,0,0,2.333075761795044,-2.217953681945801,0,2.302870988845825,-2.1831448078155518,0,2.3354077339172363,-2.1722476482391357,0,2.261728525161743,-2.178154468536377,0,1.8503494262695312,-1.861374855041504', 'val_covid_ct_scan_2,0,0,0.5681214332580566,-0.9391226768493652', 'val_covid_ct_scan_3,0,0,2.2389771938323975,-2.0930824279785156', 'val_covid_ct_scan_4,0,0,2.1063385009765625,-2.0496203899383545,0,2.1194629669189453,-2.070901393890381', 'val_covid_ct_scan_5,0,0,2.031270742416382,-2.0061323642730713,0,1.9994810819625854,-2.0244393348693848,0,2.121622085571289,-2.0909082889556885,0,2.0387184619903564,-2.0668795108795166,0,0.5567721128463745,-0.7442511916160583', 'val_covid_ct_scan_6,0,0,2.767399311065674,-2.4639060497283936,0,2.846067428588867,

In [9]:
y_true = []
y_pred = [] 
y_prob = [] 

threshold = 0.3 

covid_ids = [] 
non_covid_ids = [] 

for l in lines: 
    l_splits = l.split(',') 
    #print(l_splits)    
    #change covid class label to 1 
    true = int(l_splits[1])
    y_true.append(true)
    
    num_preds = int((len(l_splits)-2)/3)
    prob0s = [] 
    preds = [] 
    for i in range(num_preds):        
        pred = int(l_splits[2+3*i])        
        s0 = float(l_splits[2+3*i+1])
        s1 = float(l_splits[2+3*i+2])        
        prob0 = np.exp(s0)/(np.exp(s0)+np.exp(s1)) 
        preds.append(pred)
        prob0s.append(prob0)
       
    
    prob0s = np.asarray(prob0s)    
    prob0s_avg = np.mean(prob0s)    
    pred2 = (prob0s_avg < threshold).astype(np.uint8) 
    #print(true, pred2, preds, prob0s)
    y_pred.append(pred2) 
    y_prob.append(prob0s_avg)   
    #input('dbg')
    
    if pred2==0:
        covid_ids.append(l_splits[0])
    else: 
        non_covid_ids.append(l_splits[0])
    
    
y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
y_prob = np.asarray(y_prob)

print(len(y_true),len(y_pred))

F1_1 = f1_score(y_true, y_pred, average='micro')
F1_2 = f1_score(y_true, y_pred, average='macro')
print(F1_1,F1_2)

374 374
0.9171122994652406 0.9149968840500018


In [48]:
fp = open('covid-embedding.csv','w')
len_covid = len(covid_ids) 
for i in range(len_covid):
    if i<len_covid-1:
        fp.write('{},'.format(covid_ids[i]))
    else:
        fp.write('{}'.format(covid_ids[i]))
fp.close() 


fp = open('non-covid-embedding.csv','w')
len_non_covid = len(non_covid_ids) 
for i in range(len_non_covid):
    if i<len_non_covid-1:
        fp.write('{},'.format(non_covid_ids[i]))
    else:
        fp.write('{}'.format(non_covid_ids[i]))
fp.close() 